# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import matplotlib.pyplot as plt #The matplotlib visuailzation library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'week52change': 1.204291,
 'week52high': 469.9,
 'week52low': 202.52,
 'marketcap': 1924947099243,
 'employees': 143372,
 'day200MovingAvg': 313.89,
 'day50MovingAvg': 380,
 'float': 4443797050,
 'avg10Volume': 47142007.5,
 'avg30Volume': 37772632.1,
 'ttmEPS': 13.8983,
 'ttmDividendRate': 3.28,
 'companyName': 'Apple, Inc.',
 'sharesOutstanding': 4421165522,
 'maxChangePercent': 449.0185,
 'year5ChangePercent': 2.921,
 'year2ChangePercent': 1.192,
 'year1ChangePercent': 1.274,
 'ytdChangePercent': 0.5079,
 'month6ChangePercent': 0.3996,
 'month3ChangePercent': 0.4673,
 'month1ChangePercent': 0.2015,
 'day30ChangePercent': 0.2587,
 'day5ChangePercent': 0.02,
 'nextDividendDate': '2020-08-17',
 'dividendYield': 0.007336894337093787,
 'nextEarningsDate': '2020-10-25',
 'exDividendDate': '2020-08-04',
 'peRatio': 34.45,
 'beta': 1.160665118447936}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course.

In [4]:
data['year1ChangePercent']

1.274

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,99.64,0.448200,N/A
1,AAL,13.42,-0.568000,N/A
2,AAP,152.97,0.055200,N/A
3,AAPL,455.85,1.294000,N/A
4,ABBV,96.75,0.455700,N/A
...,...,...,...,...
500,YUM,95.25,-0.232361,N/A
501,ZBH,140.32,0.003299,N/A
502,ZBRA,286.51,0.351758,N/A
503,ZION,33.80,-0.204444,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DXCM,449.80,1.941147,N/A
1,NVDA,451.36,1.853632,N/A
2,AMD,87.19,1.544782,N/A
3,CARR,29.16,1.456216,N/A
4,AAPL,455.85,1.294000,N/A
5,REGN,628.61,1.012226,N/A
6,WST,276.81,0.913079,N/A
7,PYPL,198.66,0.900091,N/A
8,LRCX,385.07,0.858768,N/A
9,QRVO,132.40,0.858245,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy.

Since we've already done this, try to complete the following two code cells without watchign me do it first!

In [8]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,Number Of Shares to Buy
0,DXCM,449.80,1.941147,N/A,43.0
1,NVDA,451.36,1.853632,N/A,43.0
2,AMD,87.19,1.544782,N/A,224.0
3,CARR,29.16,1.456216,N/A,672.0
4,AAPL,455.85,1.294000,N/A,43.0
5,REGN,628.61,1.012226,N/A,31.0
6,WST,276.81,0.913079,N/A,70.0
7,PYPL,198.66,0.900091,N/A,98.0
8,LRCX,385.07,0.858768,N/A,50.0
9,QRVO,132.40,0.858245,N/A,148.0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Price Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,HQM Score,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Price Return Percentile
0,A,100.98,0.459000,N/A,N/A,0.099000,N/A,N/A,0.183100,N/A,0.252300,N/A
1,AAL,13.16,-0.556800,N/A,N/A,0.096000,N/A,N/A,-0.550000,N/A,0.378600,N/A
2,AAP,154.67,0.056500,N/A,N/A,0.117300,N/A,N/A,0.146700,N/A,0.272400,N/A
3,AAPL,450.94,1.245000,N/A,N/A,0.200700,N/A,N/A,0.402300,N/A,0.468100,N/A
4,ABBV,95.62,0.452300,N/A,N/A,-0.064700,N/A,N/A,0.006800,N/A,0.106300,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,92.40,-0.227674,N/A,N/A,0.050897,N/A,N/A,-0.122251,N/A,0.056082,N/A
501,ZBH,142.94,0.003269,N/A,N/A,0.180223,N/A,N/A,-0.122202,N/A,0.148843,N/A
502,ZBRA,296.21,0.344303,N/A,N/A,0.105428,N/A,N/A,0.143485,N/A,0.172415,N/A
503,ZION,34.54,-0.202708,N/A,N/A,0.067878,N/A,N/A,-0.263157,N/A,0.100722,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe.

In [26]:
for row in hqm_dataframe.index:
    hqm_dataframe.loc[row, 'One-Month Return Percentile'] = stats.percentileofscore(hqm_dataframe['One-Year Price Return'], hqm_dataframe.loc[row, 'One-Month Price Return'])

hqm_dataframe.sort_values('One-Month Price Return')

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,HQM Score,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Price Return Percentile
181,FE,29.47,-0.348138,N/A,N/A,-0.288451,16.831683,N/A,-0.430918,N/A,-0.275067,N/A
242,INTC,49.96,0.018437,N/A,N/A,-0.182572,26.732673,N/A,-0.287192,N/A,-0.198420,N/A
479,WDC,38.00,-0.344655,N/A,N/A,-0.136177,30.891089,N/A,-0.464259,N/A,-0.166265,N/A
328,NCLH,14.16,-0.746380,N/A,N/A,-0.130823,31.881188,N/A,-0.731818,N/A,0.127065,N/A
357,OXY,16.00,-0.687940,N/A,N/A,-0.110450,34.257426,N/A,-0.644995,N/A,0.029848,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
216,HCA,133.63,0.029070,N/A,N/A,0.373222,85.742574,N/A,-0.116483,N/A,0.213294,N/A
314,MOS,17.90,-0.265456,N/A,N/A,0.423190,87.524752,N/A,-0.138856,N/A,0.455362,N/A
465,VAR,179.18,0.546352,N/A,N/A,0.454363,89.108911,N/A,0.226264,N/A,0.476587,N/A
31,AMD,88.87,1.536012,N/A,N/A,0.615010,95.841584,N/A,0.649720,N/A,0.603283,N/A


## Calculating the HQM Score